In [39]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')
FeatureEngineering3 = FeatureEngineering3('SK_ID_CURR')

In [35]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [29]:
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

In [3]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 135)
apps_all after merge with all shape: (356255, 449)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [ ]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

In [6]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,1,0,0,1


In [7]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [8]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 589 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 579 features are remained after removing non-informative features
65 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 514 features are remained after removing features not interesting for LightGBM classifier
---=> 32 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 546 features

---=> df final shape: (356255, 546)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MIN_high_risk,POS_M20MONTHS_BALANCE_MIN_low_risk,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188,...,0,1,1,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3039,...,0,1,0,1,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038,...,0,1,0,1,0,0,0,0,0,0


In [11]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MIN_high_risk,POS_M20MONTHS_BALANCE_MIN_low_risk,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188,...,0,1,1,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3039,...,0,1,0,1,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038,...,0,1,0,1,0,0,0,0,0,0


In [12]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MIN_high_risk,POS_M20MONTHS_BALANCE_MIN_low_risk,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2329,-5168.0,...,0,0,0,0,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4469,-9120.0,...,0,1,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4458,-2176.0,...,0,1,0,1,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866,-2000.0,...,0,0,0,0,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2191,-4000.0,...,0,1,1,1,0,0,0,0,0,0


In [13]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [14]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MIN_low_risk,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2329,-5168.0,...,0,0,0,0,0,0,0,0,0,0.051055
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4469,-9120.0,...,1,1,1,0,0,0,0,0,0,0.166067
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4458,-2176.0,...,1,0,1,1,0,1,0,1,0,0.043899
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866,-2000.0,...,0,0,0,1,0,1,0,1,0,0.056542
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2191,-4000.0,...,1,1,1,0,0,0,0,0,0,0.183168


In [15]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [16]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2329,-5168.0,...,0,0,0,0,0,0,0,0,0.051055,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4469,-9120.0,...,1,1,0,0,0,0,0,0,0.166067,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4458,-2176.0,...,0,1,1,0,1,0,1,0,0.043899,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866,-2000.0,...,0,0,1,0,1,0,1,0,0.056542,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2191,-4000.0,...,1,1,0,0,0,0,0,0,0.183168,0.181109


In [17]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [18]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2329,-5168.0,...,0,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4469,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4458,-2176.0,...,1,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866,-2000.0,...,0,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2191,-4000.0,...,1,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [19]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188,...,0,0,0,0,0,0,0,0.010645,0.027249,0.012034
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3039,...,1,1,0,1,0,1,0,0.059492,0.029480,0.029856
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038,...,1,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [20]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2329,-5168.0,...,0,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4469,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4458,-2176.0,...,1,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866,-2000.0,...,0,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2191,-4000.0,...,1,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [21]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [22]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637,-3648.0,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188,-1186.0,...,0,0,0,0,0,0,0,0.010645,0.027249,0.012034
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225,-4260.0,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3039,-9832.0,...,1,1,0,1,0,1,0,0.059492,0.029480,0.029856
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038,-4312.0,...,1,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [23]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577538,0.142129,-0.478095,-0.166146,-0.507236,-0.149780,1.506880,-0.456215,0.379883,...,1.042294,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,5.053162,0.986709,0.860543
100003,0.0,-0.577538,0.426792,1.725450,0.592666,1.599974,-1.252930,-0.166821,-0.460115,1.079102,...,-0.959422,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.839862,-0.548398,-0.659948
100004,0.0,-0.577538,-0.427196,-1.152888,-1.404649,-1.091895,-0.783203,-0.689509,-0.453299,0.206055,...,-0.959422,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.334248,-0.507271,-0.508977
100006,0.0,-0.577538,-0.142533,-0.711430,0.177865,-0.653401,-0.928711,-0.680114,-0.473217,-1.375000,...,1.042294,1.693306,-0.138613,1.613795,0.0,1.613795,0.0,-0.271209,-0.523054,-0.471789
100007,0.0,-0.577538,-0.199466,-0.213734,-0.361748,-0.068741,0.562988,-0.892535,-0.473210,0.191284,...,1.042294,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.077109,-0.317542,-0.420660


In [24]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577538,0.142129,-0.478095,-0.166146,-0.507236,-0.149780,1.506880,-0.456215,...,1.042294,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,5.053162,0.986709,0.860543
1,100003,0.0,-0.577538,0.426792,1.725450,0.592666,1.599974,-1.252930,-0.166821,-0.460115,...,-0.959422,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.839862,-0.548398,-0.659948
2,100004,0.0,-0.577538,-0.427196,-1.152888,-1.404649,-1.091895,-0.783203,-0.689509,-0.453299,...,-0.959422,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.334248,-0.507271,-0.508977
3,100006,0.0,-0.577538,-0.142533,-0.711430,0.177865,-0.653401,-0.928711,-0.680114,-0.473217,...,1.042294,1.693306,-0.138613,1.613795,0.0,1.613795,0.0,-0.271209,-0.523054,-0.471789
4,100007,0.0,-0.577538,-0.199466,-0.213734,-0.361748,-0.068741,0.562988,-0.892535,-0.473210,...,1.042294,-0.590561,-0.138613,-0.619657,0.0,-0.619657,0.0,-0.077109,-0.317542,-0.420660


In [25]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 686.24 MB
Memory usage of optimization is 675294284.00 MB
Decreased by -98404725.8%
Memory usage of dataframe is 108.68 MB
Memory usage of optimization is 106846976.00 MB
Decreased by -98309513.5%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_SUM_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577637,0.142090,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,-0.456299,...,1.041992,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,5.054688,0.986816,0.860352
1,100003,0.0,-0.577637,0.426758,1.725586,0.592773,1.599609,-1.252930,-0.166870,-0.460205,...,-0.959473,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.839844,-0.548340,-0.660156
2,100004,0.0,-0.577637,-0.427246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,-0.453369,...,-0.959473,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.334229,-0.507324,-0.508789
3,100006,0.0,-0.577637,-0.142578,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.473145,...,1.041992,1.693359,-0.138672,1.614258,0.0,1.614258,0.0,-0.271240,-0.522949,-0.471680
4,100007,0.0,-0.577637,-0.199463,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.473145,...,1.041992,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.077087,-0.317627,-0.420654


In [26]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [27]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [30]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 547) (61503, 547)
[100]	training's auc: 0.796643	valid_1's auc: 0.786582
[200]	training's auc: 0.802929	valid_1's auc: 0.790633
[300]	training's auc: 0.807765	valid_1's auc: 0.793177
[400]	training's auc: 0.811654	valid_1's auc: 0.794429
[500]	training's auc: 0.815245	valid_1's auc: 0.795242
[600]	training's auc: 0.818575	valid_1's auc: 0.795641
[700]	training's auc: 0.821627	valid_1's auc: 0.795839
[800]	training's auc: 0.82462	valid_1's auc: 0.796128
[900]	training's auc: 0.827489	valid_1's auc: 0.796293
[1000]	training's auc: 0.830275	valid_1's auc: 0.796316
[1100]	training's auc: 0.832855	valid_1's auc: 0.796454
[1200]	training's auc: 0.835368	valid_1's auc: 0.79644
[1300]	training's auc: 0.837764	valid_1's auc: 0.796455
[1400]	training's auc: 0.840123	valid_1's auc: 0.796354
[1500]	training's auc: 0.842326	valid_1's auc: 0.796321
[1600]	training's auc: 0.844464	valid_1's auc: 0.796252
[1700]	training's auc: 0.84658	valid_1's auc

[1300]	training's auc: 0.838732	valid_1's auc: 0.792231
[1400]	training's auc: 0.841172	valid_1's auc: 0.792147
[1500]	training's auc: 0.843573	valid_1's auc: 0.792112
[1600]	training's auc: 0.845841	valid_1's auc: 0.792017
[1700]	training's auc: 0.84792	valid_1's auc: 0.79196
[1800]	training's auc: 0.850027	valid_1's auc: 0.791921
[1900]	training's auc: 0.852175	valid_1's auc: 0.791859
[2000]	training's auc: 0.854201	valid_1's auc: 0.791737
[2100]	training's auc: 0.856085	valid_1's auc: 0.791665
[2200]	training's auc: 0.857992	valid_1's auc: 0.791605
[2300]	training's auc: 0.859817	valid_1's auc: 0.791554
[2400]	training's auc: 0.861704	valid_1's auc: 0.791412
[2500]	training's auc: 0.863487	valid_1's auc: 0.79131
[2600]	training's auc: 0.865222	valid_1's auc: 0.791196
[2700]	training's auc: 0.86698	valid_1's auc: 0.791116
[2800]	training's auc: 0.868741	valid_1's auc: 0.791044
[2900]	training's auc: 0.870455	valid_1's auc: 0.790925
[3000]	training's auc: 0.872086	valid_1's auc: 0.790

In [31]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
546,set2a,840.6,37.845740
21,APPS_ANNUITY_CREDIT_RATIO,573.4,70.237454
545,pred_1b,323.0,18.920888
544,pred_1a,285.4,14.707141
241,EXT_SOURCE_3,257.2,27.734455
219,DAYS_BIRTH,248.8,25.223005
240,EXT_SOURCE_2,240.4,23.287336
2,AMT_GOODS_PRICE,229.0,18.096961
0,AMT_ANNUITY,214.8,35.856659
510,PREV_PREV_INTERESTS_RATE_MEAN,209.6,16.102795


In [32]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [33]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [34]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_OS_3-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.038155
1,100005,0.130656
2,100013,0.062050
3,100028,0.039034
4,100038,0.196963


## パラメータ・チューニング

In [40]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-07-19 00:01:17,480] A new study created in memory with name: no-name-df862cdc-6017-489c-804c-d4086b6f1239
[I 2022-07-19 00:04:30,968] Trial 0 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 241, 'min_child_samples': 165, 'min_sum_hessian_in_leaf': 0.00019932485005063924, 'feature_fraction': 0.7094011018171225, 'bagging_fraction': 0.5664677719068973, 'lambda_l1': 0.21209200170811635, 'lambda_l2': 8.511905655877966}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:04:44,978] Trial 1 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 42, 'min_child_samples': 83, 'min_sum_hessian_in_leaf': 5.416847718980566e-05, 'feature_fraction': 0.810954568275315, 'bagging_fraction': 0.6826053986920511, 'lambda_l1': 0.05245187304250684, 'lambda_l2': 0.3898667165266898}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:04:52,133] Trial 2 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 9, 'min_child_

[I 2022-07-19 00:20:24,923] Trial 21 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 195, 'min_child_samples': 196, 'min_sum_hessian_in_leaf': 0.0003768422025570726, 'feature_fraction': 0.5456476989928615, 'bagging_fraction': 0.8554006924480037, 'lambda_l1': 0.27500823611806446, 'lambda_l2': 3.928215620285373}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:20:52,737] Trial 22 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 203, 'min_child_samples': 116, 'min_sum_hessian_in_leaf': 0.00021110598692443303, 'feature_fraction': 0.7234463627109076, 'bagging_fraction': 0.5839528073055456, 'lambda_l1': 0.18545783220058185, 'lambda_l2': 4.088966166379457}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:20:54,278] Trial 23 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 206, 'min_child_samples': 199, 'min_sum_hessian_in_leaf': 0.00019330138731301716, 'feature_fraction': 0.8817588091673014, '

[I 2022-07-19 00:35:47,312] Trial 43 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 188, 'min_child_samples': 188, 'min_sum_hessian_in_leaf': 0.00012303548020248576, 'feature_fraction': 0.8944851161023809, 'bagging_fraction': 0.8154160809247433, 'lambda_l1': 0.1622093753758021, 'lambda_l2': 0.6221773383201302}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:35:48,144] Trial 42 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 188, 'min_child_samples': 184, 'min_sum_hessian_in_leaf': 0.00011658990279733586, 'feature_fraction': 0.896091230130897, 'bagging_fraction': 0.8161297461325123, 'lambda_l1': 0.40905712107894415, 'lambda_l2': 3.3024075755899083}. Best is trial 0 with value: 0.7964578680641743.
[I 2022-07-19 00:36:01,674] Trial 44 finished with value: 0.7964578680641743 and parameters: {'num_leaves': 213, 'min_child_samples': 123, 'min_sum_hessian_in_leaf': 0.00010563083772892929, 'feature_fraction': 0.7245912200949765, 

In [41]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7965


{'num_leaves': 241,
 'min_child_samples': 165,
 'min_sum_hessian_in_leaf': 0.00019932485005063924,
 'feature_fraction': 0.7094011018171225,
 'bagging_fraction': 0.5664677719068973,
 'lambda_l1': 0.21209200170811635,
 'lambda_l2': 8.511905655877966}

In [42]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 30,
 'min_child_samples': 70,
 'min_sum_hessian_in_leaf': 0.00019932485005063924,
 'feature_fraction': 0.7094011018171225,
 'bagging_fraction': 0.5664677719068973,
 'lambda_l1': 0.21209200170811635,
 'lambda_l2': 8.511905655877966,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.02,
 'max_bin': 400,
 'max_depth': -1,
 'subsample': 1.0,
 'subsample_freq': 1,
 'colsample_bytree': 0.05,
 'min_split_gain': 0.5,
 'reg_alpha': 0.0,
 'reg_lambda': 100,
 'n_estimators': 10000}

In [43]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246008, 547) (61503, 547)
[LightGBM] [Warning] feature_fraction is set=0.7094011018171225, colsample_bytree=0.05 will be ignored. Current value: feature_fraction=0.7094011018171225
[LightGBM] [Warning] lambda_l1 is set=0.21209200170811635, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21209200170811635
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00019932485005063924, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00019932485005063924
[LightGBM] [Warning] bagging_fraction is set=0.5664677719068973, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5664677719068973
[LightGBM] [Warning] lambda_l2 is set=8.511905655877966, reg_lambda=100 will be ignored. Current value: lambda_l2=8.511905655877966
[100]	training's auc: 0.804767	valid_1's auc: 0.795659
[200]	training's auc: 0.811101	valid_1's auc: 0.796051
[300]	training's auc: 0.817213	valid_1's auc: 0.796016
[400]	training's 

[auc] tr:0.811564, va0.7964
-------------------- 4 --------------------
(246009, 547) (61502, 547)
[LightGBM] [Warning] feature_fraction is set=0.7094011018171225, colsample_bytree=0.05 will be ignored. Current value: feature_fraction=0.7094011018171225
[LightGBM] [Warning] lambda_l1 is set=0.21209200170811635, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21209200170811635
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00019932485005063924, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00019932485005063924
[LightGBM] [Warning] bagging_fraction is set=0.5664677719068973, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5664677719068973
[LightGBM] [Warning] lambda_l2 is set=8.511905655877966, reg_lambda=100 will be ignored. Current value: lambda_l2=8.511905655877966
[100]	training's auc: 0.805576	valid_1's auc: 0.792812
[200]	training's auc: 0.811685	valid_1's auc: 0.793101
[300]	training's auc: 0.817898	valid_1's auc

In [44]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_OS_3-2.csv", index=None)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.
(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.035072
1,100005,0.112501
2,100013,0.052645
3,100028,0.037694
4,100038,0.189779
